<a href="https://colab.research.google.com/github/Keerthana-Uppanda/Multi_Agent_Ai_System/blob/main/MultiagentAISystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install and Start Redis
!apt-get install redis-server -y
!pip install redis
!redis-server --daemonize yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libjemalloc2 liblua5.1-0 liblzf1 lua-bitop lua-cjson redis-tools
Suggested packages:
  ruby-redis
The following NEW packages will be installed:
  libjemalloc2 liblua5.1-0 liblzf1 lua-bitop lua-cjson redis-server
  redis-tools
0 upgraded, 7 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,273 kB of archives.
After this operation, 5,725 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjemalloc2 amd64 5.2.1-4ubuntu1 [240 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 liblua5.1-0 amd64 5.1.5-8.1build4 [99.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 liblzf1 amd64 3.6-3 [7,444 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 lua-bitop amd64 1.0.2-5 [6,680 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 lua

In [ ]:
import redis
import json
import uuid
class MemoryStore:
    def __init__(self):
        self.db = redis.Redis(host='localhost', port=6379, db=0)

    def log_entry(self, entry):
        key = str(uuid.uuid4())
        entry["timestamp"] = str(uuid.uuid1())
        self.db.set(key, json.dumps(entry))

    def view_all(self):
        logs = []
        for key in self.db.keys():
            logs.append(json.loads(self.db.get(key)))
        return logs

In [ ]:
import json
def detect_format(data):
    try:
        json.loads(data)
        return "JSON"
    except:
        if "From:" in data:
            return "Email"
        return "PDF"

In [ ]:
def classify_intent(data):
    data_lower = data.lower()
    if "invoice" in data_lower:
        return "Invoice"
    elif "rfq" in data_lower:
        return "RFQ"
    elif "complaint" in data_lower:
        return "Complaint"
    elif "regulation" in data_lower:
        return "Regulation"
    return "Unknown"

In [ ]:
class FallbackAgent:
    def __init__(self, memory):
        self.memory = memory
    def handle(self, data, reason="Unclassified"):
        print("Fallback triggered. Reason:", reason)
        self.memory.log_entry({
            "type": "Fallback",
            "reason": reason,
            "raw_data": data
        })

In [ ]:
class JSONAgent:
    def __init__(self, memory):
        self.memory = memory
    def handle(self, data):
        try:
            json_obj = json.loads(data)
            print("JSON Parsed:", json_obj)
            self.memory.log_entry({
                "type": "JSON",
                "data": json_obj,
                "status": "Processed"
            })
        except Exception as e:
            print("JSON Error:", str(e))

In [ ]:
class EmailAgent:
    def __init__(self, memory):
        self.memory = memory

    def handle(self, data):
        lines = data.splitlines()
        sender = ""
        urgency = "Normal"
        for line in lines:
            if line.lower().startswith("from:"):
                sender = line.split(":", 1)[1].strip()
            if "urgent" in line.lower():
                urgency = "High"

        self.memory.log_entry({
            "type": "Email",
            "sender": sender,
            "urgency": urgency,
            "content": data
        })
        print(f"Email from {sender}, Urgency: {urgency}")

In [ ]:
class ClassifierAgent:
    def __init__(self, memory):
        self.memory = memory
    def classify(self, data):
        # Use the functions defined in the notebook directly
        file_format = detect_format(data)
        intent = classify_intent(data)
        self.memory.log_entry({
            "type": "Classifier",
            "format": file_format,
            "intent": intent
        })
        return file_format, intent

In [ ]:
# Instantiate all agents
memory = MemoryStore()
classifier = ClassifierAgent(memory)
json_agent = JSONAgent(memory)
email_agent = EmailAgent(memory)
fallback_agent = FallbackAgent(memory)

def process_input(input_data):
    file_format, intent = classifier.classify(input_data)
    print(f"Format: {file_format}, Intent: {intent}")

    if file_format == "Unknown":
        fallback_agent.handle(input_data, reason="Unknown Format")
    elif intent == "Unknown":
        fallback_agent.handle(input_data, reason="Unknown Intent")
    else:
        if file_format == "JSON":
            json_agent.handle(input_data)
        elif file_format == "Email":
            email_agent.handle(input_data)
        elif file_format == "PDF":
            print("PDF simulated handling.")
            memory.log_entry({
                "type": "PDF",
                "note": "Simulated PDF handler",
                "content": input_data
            })
        else:
            fallback_agent.handle(input_data, reason="Unhandled Format")

In [ ]:
json_ip = '''
{
    "id": "202245",
    "amount": 5000,
    "email": "uppandakeerthana@gmail.com",
    "type": "invoice"
}
'''
process_input(json_ip)

Format: JSON, Intent: Invoice
JSON Parsed: {'id': '202245', 'amount': 5000, 'email': 'uppandakeerthana@gmail.com', 'type': 'invoice'}


In [ ]:
email_input = """
From: uppandakeerthana@gmail.com
Subject: Complaint - Product not delivered
Dear Support,

I placed an order two weeks ago and still haven't received the product. Please investigate and get back to me urgently.

Regards,
Keerthana
"""
process_input(email_input)

Format: Email, Intent: Complaint
Email from uppandakeerthana@gmail.com, Urgency: High


In [ ]:
email_input = """
From: uppandakeerthana@gmail.com
Subject: RFQ for hardware

Hello Team,

We are interested in procuring 200 laptops for our new office expansion. Kindly send us your best quotation.

Regards,
Keerthana
"""
process_input(email_input)

Format: Email, Intent: RFQ
Email from uppandakeerthana@gmail.com, Urgency: Normal


In [ ]:
pdf_input = """
Subject: New Workplace Regulation 2024-C

As per regulation 2024-C, all contractors must submit compliance certificates by 30 June 2025.

Issued by: Health and Safety Board
"""
process_input(pdf_input)

Format: PDF, Intent: Regulation
PDF simulated handling.


In [ ]:
print("\n MEMORY LOGS:")
for entry in memory.view_all():
    print(entry)


 MEMORY LOGS:
{'type': 'Classifier', 'format': 'PDF', 'intent': 'Regulation', 'timestamp': 'e34e6f3a-3bba-11f0-9c5b-0242ac1c000c'}
{'type': 'Classifier', 'format': 'Email', 'intent': 'RFQ', 'timestamp': '60cbc828-3bba-11f0-9c5b-0242ac1c000c'}
{'type': 'Fallback', 'reason': 'Unknown Intent', 'raw_data': '\nHey team, just checking in to see how everyone’s doing. No rush — just saying hi!\n', 'timestamp': '8b2b5346-3bb9-11f0-9c5b-0242ac1c000c'}
{'type': 'Classifier', 'format': 'JSON', 'intent': 'Invoice', 'timestamp': 'cb326fe2-3bb9-11f0-9c5b-0242ac1c000c'}
{'type': 'Email', 'sender': 'uppandakeerthana@gmail.com', 'urgency': 'High', 'content': "\nFrom: uppandakeerthana@gmail.com\nSubject: Complaint - Product not delivered\nDear Support,\n\nI placed an order two weeks ago and still haven't received the product. Please investigate and get back to me urgently.\n\nRegards,\nKeerthana\n", 'timestamp': '8f006bd6-3bba-11f0-9c5b-0242ac1c000c'}
{'type': 'Email', 'sender': 'uppandakeerthana@gmail.